In [1]:
%run '0.0_init_configuration.ipynb'

In [2]:
#Importamos activo de yfinace BTC-USD
activo = 'BTC-USD'
df = yf.Ticker(activo)
df = df.history(start='2014-01-01', end='2025-04-07')
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,0.0,0.0
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,0.0,0.0
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,0.0,0.0
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,0.0,0.0
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,0.0,0.0
...,...,...,...,...,...,...,...
2025-04-02 00:00:00+00:00,85180.609375,88466.953125,82343.539062,82485.710938,47584398470,0.0,0.0
2025-04-03 00:00:00+00:00,82487.476562,83909.296875,81282.101562,83102.828125,36852112080,0.0,0.0
2025-04-04 00:00:00+00:00,83100.250000,84696.148438,81670.750000,83843.804688,45157640207,0.0,0.0


In [3]:
#Definir funciòn para calcular el rendimiento de la estrategia
def backtest_strategy(short_window, long_window):
    df['EMA_short'] = ta.EMA(df['Close'], timeperiod=short_window)
    df['EMA_long'] = ta.EMA(df['Close'], timeperiod=long_window)
    df['Signal'] = np.where(df['EMA_short'] > df['EMA_long'], 1, -1)
    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    return (df['Strategy_Returns'] + 1).cumprod().iloc[-1] - 1 # Retorno Total

In [4]:
# Definir el rango de parámetros a probar
short_windows = range(10, 100, 1) # De 10 a 100 días
long_windows = range(10, 200, 1) # De 10 a 200 días

In [5]:
# Probar todas las combinaciones de parámetros
best_performance = -np.inf
best_params = (None, None)

In [6]:
for short_window in short_windows:
    for long_window in long_windows:
        if short_window >= long_window:
            continue # Evitar combinaciones donde la media corta sea mayor o igual a la media larga
        performance = backtest_strategy(short_window, long_window)
        if performance > best_performance:
            best_performance = performance
            best_params = (short_window, long_window)

print(f"Mejores parámetros: EMA_corta={best_params[0]}, EMA_larga={best_params[1]}")
print(f"Mejor Rendimiento: {best_performance:.2%}")

Mejores parámetros: EMA_corta=22, EMA_larga=68
Mejor Rendimiento: 35456.76%
